In [1]:
from __future__ import print_function
import ipywidgets as widgets 
import matplotlib.pyplot as plt
import numpy as np

# JC Model

### Job Creator parameters

$r = instructionCount*JO.rate + bandwidthUsage*JO.rate$ : The reward the JC is willing to pay to have a job executed.

$pID = 1$ : The probability that JC will identify a job as class 1, 2, or junk.

$p_{v1}$ : probability the JC verifies answers from class 1. 

$p_{v2}$ : probability the JC verifies answers from class 2. This probability is 0 because it can just go straight to mediation. 

$vc$ : cost to JC to verify result.

$b$ : the value or benefit JC gets when a job is performed correctly.

$matchPrice$ : the amount JC pays to Solver for finding a suitable RP. 

$mediatorPrice$ : The amount JC pays to mediator for being available. 

$p_1$ : probability that job returns answer in class $a1$

$p_2 = 1-p_1$ : probability that job returns answer in class $a2$

$deposit = price\times PR\times n$ : the amount the Job Creator must post as a security deposit in order to get matched. It multiplied by the $n$, which as a reminder is the number of times the mediator will replicate a job. 

In [2]:
def JCU(n,PR,NDPR,  b,r,p1,pID,idc,pv1,vc1,pv2,vc2,gasCost,  pq):
    
    p2 = 1-p1
    
    NDdeposit = r * NDPR * n
    
    honest = pID*p1*pv1*pq * (b-r-vc1-idc)
    
    # ----------------------------------------------------------
    # 1-pq is likely not p1 or p2 so deal with it separately?
    # ----------------------------------------------------------
#     DRP_Gain   = p1*pv1*(1-pq) * (p1**n) * (mb + r - vc1)     
#     DRP_Lose   = p1*pv1*(1-pq) * (1-p1**n) * ( - NDdeposit - vc1)    
    
    lazy   = pID*p1*(1-pv1)*pq * (b - r - idc)
    
#     unlucky = p1*(1-pv1)*(1-pq) * (-r)
    
    normal = honest + lazy
    
    # ----------------------------------------------------
    # pv2 will be 0. 
    # ----------------------------------------------------
#     CNDJC_HRP_Gain = p2*pv2*pq * (p1**n) * (b + r - vc2)     
#     CNDJC_HRP_Lose = p2*pv2*pq * (1-p1**n) * (b - NDdeposit - vc2)
    
#     CNDJC_DRP_Gain = p2*pv2*(1-pq) * (p1**n) * (mb + r - vc2)     
#     CNDJC_DRP_Lose = p2*pv2*(1-pq) * (1-p1**n) * ( - NDdeposit - vc2)
    
    NDJC_HRP_Gain = pID*p2*(1-pv2)*pq   * (p1**n) * (b + r - idc)    # gasCost is refunded  
    NDJC_HRP_Lose = pID*p2*(1-pv2)*(pq) * (1-p1**n) * (b - NDdeposit - gasCost - idc)
    
    ND = NDJC_HRP_Gain + NDJC_HRP_Lose
    
    # ----------------------------------------------------------
    # 1-pq is likely not p1 or p2 so deal with it separately?
    # ----------------------------------------------------------
#     NDJC_DRP_Gain = p2*(1-pv2)*(1-pq) * (p1**n) * (mb + r)  
#     NDJC_DRP_Lose = p2*(1-pv2)*(1-pq) * (p1**n) * ( - NDdeposit)
    
    DRP_Gain = pID*(1-pq) * (p1**n) * (mb + r - idc)
    DRP_Lose = pID*(1-pq) * (1-p1**n) * (- NDdeposit - gasCost - idc)  
    DRP_Fixd = pID*(1-pq) * (-r - idc)
    
    if p1 != 1:
        '''JC is non-deterministic'''
        DRP = max(DRP_Gain+DRP_Lose, DRP_Fixd)
    else: 
        '''JC is deterministic'''
        DRP = DRP_Fixd           
    
    noID = (1-pID) * (-r)
    
    return normal + ND + DRP + noID

    

In [13]:
x,step = np.linspace(0,1,51,retstep=True)

def plotUtil( n,PR,NDPR,  b,r,pID,idc,pv1,vc1,pv2,vc2,  pq):
  
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    
    print("step size: %s" %step)
    
    yJND = list(map(lambda p1: JCU(n,PR,NDPR, b=b,r=r,p1=p1,pID=1,idc=0,pv1=1,vc1=0,pv2=0,vc2=0,gasCost=JCgasCost, pq=pq),x))
    yJ   = list(map(lambda p1: JCU(n,PR,NDPR, b=b,r=r,p1=1,pID=1,idc=0,pv1=1,vc1=0,pv2=0,vc2=0,gasCost=JCgasCost, pq=pq),x))
                
#     yRND = list(map(lambda p1: RPNDU(M_n,M_PR,M_NDPR, p1,price,p=p, q=q,roi=roi,I=Insurance),x))
#     yRNDD = list(map(lambda p1: RPNDUD(M_n,M_PR,M_NDPR,p1,price,p=p,rcj=rcj,q=q,roi=roi,I=Insurance),x))
    plt.plot(x,yJ, label="JU")
    plt.plot(x,yJND, label="JUND")
#     plt.plot(x,yRND, label="RUND")
#     plt.plot(x,yRNDD, label="RUNDD")
    plt.xlabel('P(a1)')
    plt.ylabel('Util') 
    
    plt.legend()
    plt.show

In [14]:


# Mediator parameters
wn = widgets.IntSlider(min=0,max=10,step=1,value=4, description='n',continuous_update=False)
wPR = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='PR', continuous_update=False) #Penalty Rate
wNDPR = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='NDPR',continuous_update=False)#Non-deterministic Penalty rate

# JC parameters
wb = widgets.IntSlider(min=0,max=10000,step=1,value=1, description='b',continuous_update=False)
wr = widgets.IntSlider(min=0,max=10000,step=1,value=1, description='r',continuous_update=False)
wpID = widgets.FloatSlider(min=0,max=1,step=.1,value=1, description='P(ID)', continuous_update=False)
widc = widgets.IntSlider(min=0,max=1000,step=1,value=1, description='idc',continuous_update=False)
wpv1 = widgets.FloatSlider(min=0,max=1,step=.1,value=1, description='P(verify 1)', continuous_update=False) # how often JC verifies
wvc1 = widgets.FloatSlider(min=0,max=100,step=.1,value=0, description='vc1',continuous_update=False) # cost for JC to verify
wpv2 = widgets.FloatSlider(min=0,max=1,step=.1,value=0, description='P(verify 2)', continuous_update=False) # how often JC verifies
wvc2 = widgets.FloatSlider(min=0,max=100,step=.1,value=0, description='vc2',continuous_update=False) # cost for JC to verify

mb = 0
JCgasCost = 0
JCmatchCost = 1
JCmediatorCost = 1

# RP parameters
wpq = widgets.FloatSlider(min=0,max=1,step=.1,value=1,description='P(run)', continuous_update=False) # probability that RP runs correctly
wroi = widgets.FloatSlider(min=0,max=1,step=.01,value=.5,description='roi', continuous_update=False) # what percent of reward is profit
winsurance = widgets.FloatSlider(min=0,max=1,step=.01,value=0 ,description='I',continuous_update=False) # % of deposit that can be recovered because execution is so cheap.
wcd= widgets.FloatSlider(min=0,max=1,step=.1,value=0, description='rcj',continuous_update=False) # cost for RP to generate fake job response

RPgasCost = 0
RPmatchCost = 1
RPmediatorCost = 1

# ui = widgets.HBox([wn,wPR,wNDPR,wPrice,wp,wcj,wq,wroi,winsurance])
ui1 = widgets.HBox([wn,wPR,wNDPR])
ui2 = widgets.HBox([wb,wr,wpID,widc])
ui22 = widgets.HBox([wpv1,wvc1,wpv2,wvc2])
ui3 = widgets.HBox([wpq,wroi,winsurance,wcd])

plotParams = {'n': wn,
              'PR':wPR,
              'NDPR':wNDPR,
              'b':wb,
              'r':wr,
              'pID':wpID,
              'idc':widc,
              'pv1':wpv1,
              'vc1':wvc1,
              'pv2':wpv2,
              'vc2':wvc2,
              'pq': wpq}



In [15]:
interactive_plot = widgets.interactive_output(plotUtil,plotParams);
# output = interactive_plot.children[-1]
# output.layout.height = '900px'
display(interactive_plot,ui1,ui2,ui22,ui3)

Output()